In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = smc.ParticleTrace(1)

In [4]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)
ssm_step.condition(trace=generative, observations=None)

In [6]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta = ssm_step(zs[:, t], mu, sigma, delta, t)

In [7]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [8]:
num_particles = 50

In [9]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [10]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [11]:
samples, elbos, inference = smc.particle_mh(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

In [12]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 50]), 'sigma': Normal([torch.cuda.FloatTensor of size 50]), 'delta': Normal([torch.cuda.FloatTensor of size 50]), 'Z_0': Normal([torch.cuda.FloatTensor of size 50]), 'Z_1': Normal([torch.cuda.FloatTensor of size 50]), 'X_1': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 50]), 'X_2': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 50]), 'X_3': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 50]), 'X_4': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 50]), 'X_5': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 50]), 'X_6': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 50]), 'X_7': Normal([torch.cuda.FloatTensor of size 50x1x1]), 'Z_8': Normal([torch.cuda.F

In [13]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [14]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 2.030367
SMC MSE at time 2: 0.008856
SMC MSE at time 3: 1.374140
SMC MSE at time 4: 4.909276
SMC MSE at time 5: 2.142078
SMC MSE at time 6: 1.666680
SMC MSE at time 7: 0.576847
SMC MSE at time 8: 0.245671
SMC MSE at time 9: 6.614482
SMC MSE at time 10: 0.982719
SMC MSE at time 11: 0.673418
SMC MSE at time 12: 1.127214
SMC MSE at time 13: 0.391711
SMC MSE at time 14: 0.338477
SMC MSE at time 15: 2.763733
SMC MSE at time 16: 0.324593
SMC MSE at time 17: 6.443355
SMC MSE at time 18: 1.712611
SMC MSE at time 19: 0.203265
SMC MSE at time 20: 0.600090
SMC MSE at time 21: 0.910585
SMC MSE at time 22: 1.114311
SMC MSE at time 23: 1.524656
SMC MSE at time 24: 0.459511
SMC MSE at time 25: 1.657846
SMC MSE at time 26: 0.270059
SMC MSE at time 27: 5.991689
SMC MSE at time 28: 0.555166
SMC MSE at time 29: 0.461214
SMC MSE at time 30: 5.332448
SMC MSE at time 31: 0.361954
SMC MSE at time 32: 1.688008
SMC MSE at time 33: 0.515037
SMC MSE at time 34: 1.607520
SMC MSE at time 35: 5.0

In [32]:
inference, init_ssm_params = smc.variational_smc(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

07/09/2018 11:12:18 Variational SMC ELBO=-1.9560e+02 at epoch 0
07/09/2018 11:12:18 Variational SMC ELBO=-1.9560e+02 at epoch 1
07/09/2018 11:12:19 Variational SMC ELBO=-1.9560e+02 at epoch 2
07/09/2018 11:12:19 Variational SMC ELBO=-1.9560e+02 at epoch 3
07/09/2018 11:12:20 Variational SMC ELBO=-1.9560e+02 at epoch 4
07/09/2018 11:12:20 Variational SMC ELBO=-1.9560e+02 at epoch 5
07/09/2018 11:12:21 Variational SMC ELBO=-1.9560e+02 at epoch 6
07/09/2018 11:12:21 Variational SMC ELBO=-1.9560e+02 at epoch 7
07/09/2018 11:12:22 Variational SMC ELBO=-1.9560e+02 at epoch 8
07/09/2018 11:12:22 Variational SMC ELBO=-1.9560e+02 at epoch 9
07/09/2018 11:12:23 Variational SMC ELBO=-1.9560e+02 at epoch 10
07/09/2018 11:12:23 Variational SMC ELBO=-1.9560e+02 at epoch 11
07/09/2018 11:12:24 Variational SMC ELBO=-1.9560e+02 at epoch 12
07/09/2018 11:12:24 Variational SMC ELBO=-1.9560e+02 at epoch 13
07/09/2018 11:12:25 Variational SMC ELBO=-1.9560e+02 at epoch 14
07/09/2018 11:12:25 Variational SMC

In [33]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.560846
SMC MSE at time 2: 1.308327
SMC MSE at time 3: 0.198603
SMC MSE at time 4: 6.088970
SMC MSE at time 5: 2.467254
SMC MSE at time 6: 0.346366
SMC MSE at time 7: 0.720208
SMC MSE at time 8: 0.134442
SMC MSE at time 9: 7.560007
SMC MSE at time 10: 3.391081
SMC MSE at time 11: 0.127761
SMC MSE at time 12: 1.199039
SMC MSE at time 13: 0.359360
SMC MSE at time 14: 0.050611
SMC MSE at time 15: 0.310277
SMC MSE at time 16: 2.650104
SMC MSE at time 17: 1.121154
SMC MSE at time 18: 2.313074
SMC MSE at time 19: 0.209921
SMC MSE at time 20: 0.055940
SMC MSE at time 21: 1.472705
SMC MSE at time 22: 0.903467
SMC MSE at time 23: 1.809980
SMC MSE at time 24: 1.457915
SMC MSE at time 25: 1.621504
SMC MSE at time 26: 4.770899
SMC MSE at time 27: 6.050775
SMC MSE at time 28: 0.160204
SMC MSE at time 29: 0.090209
SMC MSE at time 30: 3.376993
SMC MSE at time 31: 0.208659
SMC MSE at time 32: 3.247341
SMC MSE at time 33: 0.150638
SMC MSE at time 34: 1.930040
SMC MSE at time 35: 3.6